### Run Holdout classification on bacteria datasets with ANI filter.
The holdout datasets are created  using a different function (dataset.ho_get_class_list) . This includes removing viruses with closer than 75%ANI with any holdout(test) viruses from the training data.

In [ ]:
%reload_ext autoreload
%autoreload 2
import os
import pandas as pd
import pickle
import random
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
import csv
from pathlib import Path
import sys

from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split 
from sklearn.metrics import roc_auc_score

lib_dir = '../mylibs'
if lib_dir not in sys.path:
    sys.path.append(lib_dir)
    
import vhdb as vhdb
from featureset import FeatureSet
from dataset import DataSet

#### Load the input file containing datasets to be trained/tested and load the VHDB 

In [7]:
# Input file  
subsetfile = 'inputs/Bact_holdout.csv'
label_info = pd.read_csv(subsetfile)
subsets = label_info.apply(tuple, axis=1).tolist()
print(f'{len(subsets)} datasets : {subsets[0]}')

vhdbfile = '../inputs/VHDB_25_1_2019.p'
with open(vhdbfile, 'rb') as f:
    V_H = pickle.load( f)
hosts = V_H.hosts
viruses = V_H.viruses
print (f'{len(viruses)} viruses and {len(hosts)} hosts')


# Output file for the results
out_file = Path(subsetfile).stem 
results_file = 'results/test_ANI_holdout_results.csv'
print (f'Results will be saved in: {results_file}')



24 datasets : ('Bacteroidetes', 'phylum', 'Bacteria', 'kingdom', 'DNA', 'Siphoviridae')
9199 viruses and 3006 hosts
Results will be saved in: results/test_ANI_holdout_results.csv


In [15]:
def get_confusion( y_test,y_pred):
    FN=0
    FP=0
    TP=0
    TN=0
    #missed = 0
    for y,yp in zip(y_test,y_pred):
        #print (y_true,yp,yp2,yprob)
        if y == True and yp == False:
            FN += 1
        elif y == False and yp == True:
            FP +=1
        elif y == True and yp == True:
            TP +=1
        else:
            TN +=1
   
    spec = round(TN/(TN+FP),2) if (TN + TP) else 'NA'
    sens = round(TP/(TP+FN),2) if (TP+FN) else 'NA'
    prec  = round(TP/(TP+FP),2) if (TP+FP) else 'NA'
    res  = { 'Acc':round((TP+TN)/(len(y_test)),2),'Spec':spec, 'Sens':sens,\
           'Prec':prec}
    return res

def test_prediction(fs,y_trn,y_tst):
    clf = make_pipeline(StandardScaler(),svm.SVC(kernel='linear',probability=True) )
    clf.fit(fs.X_trn, y_trn)
    y_pred = clf.predict(fs.X_tst)
    y_pred_probs= clf.predict_proba(fs.X_tst)[:,1]
    AUC =round( roc_auc_score(y_tst, y_pred_probs),3)
    results = {'AUC':AUC, 'features':fs.feature,'k':fs.k}
    confusion = get_confusion(y_tst,y_pred)
    results.update(confusion)
    return results

        
        

### Set up the different features to be tested and corresponding lists:
1. Filepaths and extensions
2. k-mer lengths
3. Lookup for symbols used in sequences

In [19]:
features = ['DNA','AA','PC','Domains']
kmer_lists = [[2,6,9], # dna 
              [3,4],           # aa
              [5,6] ,      #pc
              [0]]  
feature_sets = [f'{f}_{k}' for i,f in enumerate(features) for k in kmer_lists[i] ]
#feature_sets = ['DNA_2', 'AA_2', 'PC_2', 'Domains_0'] #test
feature_sets


['DNA_2', 'DNA_6', 'DNA_9', 'AA_3', 'AA_4', 'PC_5', 'PC_6', 'Domains_0']

In [17]:
len(subsets)

24

## Main loop
#### For each dataset, run inner loop to train and test  all feature sets  as holdout classifiers. 

In [ ]:
#[ho_pos, ho_neg,not_ho_pos,not_ho_neg]

for subset in subsets[12:20]:
    print('*****',subset)
#     # for holdout data  
    holdout = subset[-1]
    subset = subset [:-1]
    print ( holdout, subset)
    data = DataSet(subset,V_H,feature_sets= [],tt_split = 0, holdout=holdout)
    
    all_results =[]
    
    mask = data.ds['trn/tst']=='train'
    y_train = np.asarray(data.ds[mask]['y'],dtype=int)
    if len(y_train) > 40:
        print ('OK*************', len(y_train))
        data.addFeatureSets(feature_sets)
        y_test = np.asarray(data.ds[~mask]['y'],dtype=int)
        for fs in data.fs:
            results = test_prediction(fs,y_train,y_test)
            data.results2CSV (results,subset, results_file)
            all_results.append(results)
results_df = pd.DataFrame(all_results)

***** ('Bacillales', 'order', 'Firmicutes', 'phylum', 'DNA', 'Podoviridae')
Podoviridae ('Bacillales', 'order', 'Firmicutes', 'phylum', 'DNA')
before ani -not_ho pos 296 neg 305
after ani -not_ho pos 286 neg 261
Bacillales_Podoviridae [296, 305, 286, 261]
ho_pos 17 ho_neg 17 not_ho_pos 200 not_ho_neg 200
holdout  Podoviridae 434
     virus  y      refseqs trn/tst
0    10778  1  [NC_004165]    test
1  1675598  1  [NC_028782]    test
2   204090  1  [NC_004679]    test
3  1010614  1  [NC_016565]    test
4  1597966  1  [NC_028926]    test
OK************* 399
['DNA_2', 'DNA_6', 'DNA_9', 'AA_3', 'AA_4', 'PC_5', 'PC_6', 'Domains_0'] <class 'list'>
adding fs DNA_2
DNA_2 <class 'str'>
FS get sequences for length ds 432
FS get feature names DNA_2 432
FS get X len fs 16
432 432 16 (432, 16)
adding fs DNA_6
DNA_6 <class 'str'>
FS get sequences for length ds 432
FS get feature names DNA_6 432
FS get X len fs 4096
432 432 4096 (432, 4096)
adding fs DNA_9
DNA_9 <class 'str'>
FS get sequences for leng